In [ ]:
# Import glosim2 (https://github.com/epfl-cosmo/glosim2)
import sys
sys.path.insert(0, '/home/azadoks/git/glosim2')
# In my docker, use this:
# sys.path.insert(0, '/home/app/glosim2')

# Import Atoms objects
from ase import Atoms as ase_Atoms
from quippy import Atoms as quippy_Atoms

# Data management
import numpy as np
import pandas as pd
# If data is too large for memory, I suggest replacing (or augmenting)
# numpy Arrays and pandas DataFrames with dask arrays and dataframes

# SOAP
from libmatch.soap import get_Soaps
from libmatch.utils import ase2qp

# PCA
from sklearn.decomposition import PCA

DISPLAY_DATA = True

### Structure Input
---

In [ ]:
# Load structures here

# For example:
from pymatgen import MPRester, Structure
from pymatgen.io.ase import AseAtomsAdaptor
from collections import OrderedDict
material_ids = OrderedDict([
    ('diamond', 'mp-66'),  # C
    ('rocksalt', 'mp-22862'),  # NaCl
    ('cubic_perovskite', 'mp-2998'),  # BaTiO3
    ('wurtzite', 'mp-10281'),  # ZnS
    ('fcc', 'mp-23'),  # Ni
    ('bcc', 'mp-13'),  # Fe
    ('hcp', 'mp-153'),  # Mg
    ('trigonal', 'mp-782'),  # Te2Pd
    ('tetragonal', 'mp-742'),  # Ti2Cu
    ('monoclinic', 'mp-684'),  # BaS2
    ('triclinic', 'mp-9122'),  # CaP3
    ('orthorhombic', 'mp-872')  # BaSn
])
# Convert material_ids dict into a DataFrame with a column for name and material_id
material_df = pd.DataFrame({'name': material_ids.keys(), 'material_id': material_ids.values()})
print 'Before applying mpr.get_structure_by_material_id'
if DISPLAY_DATA: display(material_df)

# .apply will call its argument (in this case, mpr.get_structure_by_material_id) 
#   on each of the entries of the object on which it's called (in this case, the material_id Series)
#   (DataFrame columns are Series)
# Here, it retrieves a pymatgen Structure from Materials Project for each material_id
with MPRester('0WqdPfXxloze6T9N') as mpr:
    material_df['structure'] = material_df['material_id'].apply(mpr.get_structure_by_material_id)

print ''.join(['=']*80)
print 'After applying the query function and making a new "structure" column'
if DISPLAY_DATA: display(material_df)

### Structure Conversion
---

In [ ]:
# Convert structures to ase Atoms
material_df['ase'] = material_df['structure'].apply(AseAtomsAdaptor.get_atoms)
if DISPLAY_DATA: display(material_df)

In [ ]:
# Convert ase Atoms to quippy Atoms
material_df['quippy'] = material_df['ase'].apply(ase2qp)
if DISPLAY_DATA: display(material_df)

### Soap Calculation
---
* `atoms` (`[quippy.Atoms]`): List of quippy Atoms structures
* `nocenters` (`[int]` or `None`): List of atomic numbers to ignore as centers
* `chem_channels` (`bool`): ??
* `centerweight` (`float`): Weight of gaussian on central atom
* `gaussian_width` (`float`): Width (sigma) of gaussian 
* `cutoff` (`float`): Distance (in units of input) to cut off overlap integration
* `cutoff_transition_width` (`float`): Width of sigmoid used to smooth integration cutoff
* `nmax` (`nmax`): Number of radial basis functions
* `lmax` (`int`): Number of spherical harmonics
* `spkitMax` (`dict`): "species kit maximum", `{Z: Nmax}` over all structures in `atoms`

        spkit = {}
        for atom in all:
            atomspecies = {}
            for z in atom.z:      
                if z in atomspecies: atomspecies[z]+=1
                else: atomspecies[z] = 1

            for (z, nz) in atomspecies.iteritems():
                if z in spkit:
                    if nz>spkit[z]: spkit[z] = nz
                else:
        spkit[z] = nz

* `nprocess`: Number of subprocesses spawned (best to use number of cores unless data is very large
* `chemicalProjection` (`None` or `???`)
* `dispbar` (`bool`)
* `is_fast_average` (`bool`): Return fast average if true, full soap (per site) if false. If true, will return `OrderedDict([('AVG': SOAP)])`

In [ ]:
# default SOAP parameters
# get_Soaps(atoms, 
#           nocenters=None, chem_channels=False, 
#           centerweight=1.0, gaussian_width=0.5, 
#           cutoff=3.5, cutoff_transition_width=0.5, 
#           nmax=8, lmax=6, 
#           spkitMax=None, 
#           nprocess=1, 
#           chemicalProjection=None, 
#           dispbar=False, 
#           is_fast_average=False)

# Calculate SOAPs
material_df['soaps'] = get_Soaps(material_df['quippy'])
material_df['fast_average_soap'] = get_Soaps(material_df['quippy'], is_fast_average=True)
if DISPLAY_DATA: display(material_df)

# Use example_soap to investigate the structure of a SOAP
example_soap = material_df['soaps'][0]

In [ ]:
# Calculate average SOAPs
def get_average_soap(soaps):
    return np.mean(soaps.values(), axis=0)
material_df['average_soap'] = material_df['soaps'].apply(get_average_soap)
if DISPLAY_DATA: display(material_df)

### PCA
---

In [ ]:
# Create the PCA data

# Create a numpy array with the same number of rows as structures
#   and the same number of columns as SOAP dimensions
len_entries = len(material_df['average_soap'])
len_soap = len(material_df['average_soap'][0])
# Here, concatenate makes an len_entries * len_soap x 1 vector which
#  is reshaped into a len_entries x len_soap array
pca_array = np.concatenate(material_df['average_soap']).reshape(len_entries, len_soap)

In [ ]:
# Do a test PCA on the average_soaps

# This will do a PCA resulting in vectors of the same length (dimension) as the originals
# Do this to find the percent of variance explained by various numbers of dimensions
pca = PCA()
pca.fit(pca_array)
print(list(enumerate(pca.explained_variance_ratio_.cumsum())))

In [ ]:
# Do the final PCA

# 10 components gives practically 100% of variance explaned for the example data
pca = PCA(n_components=10) 
pca_soap = pca.fit_transform(pca_array)
if DISPLAY_DATA: material_df['pca_soap'] = pca_soap.tolist()

In [ ]:
# Save only the necessary data
material_df[['material_id', 'pca_soap']].to_json()